In [1]:
import random
from datetime import date
import numpy as np
import pandas as pd
from ortools.sat.python import cp_model

In [2]:
# Constants
DAYS = 7
WEEKDAYS_DICTIONARY = {0 : "Sun", 1 : "Mon", 2 : "Tue", 3 : "Wed", \
                       4 : "Thu", 5 : "Fri", 6 : "Sat"}
SHIFTS = 2
SHIFTS_DICTIONARY = {0 : "AM", 1 : "PM"}
PART_TIME_SHIFTS_AT_MOST = 3
PART_TIME_SHIFTS_AT_LEAST = 2
FULL_TIME_SHIFTS_AT_MOST = 6
FULL_TIME_SHIFTS_AT_LEAST = 4
EXCEL_NAME = "bartender.xlsx"

In [3]:
# Read the Excel
preferences = pd.read_excel(EXCEL_NAME, "Preferences")
number_employee_shifts = pd.read_excel(EXCEL_NAME, "Shifts Requirement")
employees = preferences["Employee Name"].last_valid_index() + 1
part_time = pd.read_excel(EXCEL_NAME, "Part Time")

In [4]:
# Make a dictionary that every employee's index is the key, value is 0 or 1, 0 means not part time, 1 is part time
is_part_time = {}
for e in range(employees):
    if part_time.iat[e, 1].lower() == "yes":
        is_part_time[e] = 1
    else:
        is_part_time[e] = 0

In [5]:
# Make preference matrix
prefer_value = [[[0 for s in range(SHIFTS)] for d in range(DAYS)] for e in range(employees)]

for e in range(employees):
    count = 1
    while count <= (DAYS * SHIFTS):
        d = (count -1) // SHIFTS
        s = (count -1) % SHIFTS
        prefer_value[e][d][s] = preferences.iat[e, count]
        count += 1

In [6]:
# Each shift require different number of employees, so below is requirement matrix
shifts_requirement = [[0 for s in range(SHIFTS)] for d in range(DAYS)]
for d in range(DAYS):
    for s in range(SHIFTS):
        shifts_requirement[d][s] = number_employee_shifts.iat[s, d+1]

In [7]:
# create instance of cp_model
model = cp_model.CpModel()

In [8]:
# Assign all the shifts to x[e,d,s] e means employee, d means days, s means morning shift or evening shift
x = {}
for e in range(employees):
    for d in range(DAYS):
        for s in range(SHIFTS):
            x[(e, d, s)] = model.NewBoolVar("x[%d,%d,%d]" % (e, d, s))

In [9]:
# Constraint 1: Each shift in a week require different number of employee
for d in range(DAYS):
    for s in range(SHIFTS):
        model.Add(sum(x[e, d, s] for e in range(employees)) == shifts_requirement[d][s])

In [10]:
# Constraint 2: No one can work two shifts in a day
for e in range(employees):
    for d in range(DAYS):
        model.Add(sum(x[e, d, s] for s in range(SHIFTS)) <= 1)

In [11]:
# Constraint 3: No one can work two consecutive shifts(avoiding today's evening shift and tomorrow's morning shift)
for d in range(DAYS):
    for e in range(employees):
        for s in range(SHIFTS):
            model.Add((x[e, d, 1] + x[e, (d+1)% DAYS, 0]) <= 1)

In [12]:
# Constraint 4: Part time employee and full time employee have different number of shifts in a week
for e in range(employees):
    if is_part_time[e] == 1:
        model.Add(sum(x[e, d, s] for d in range(DAYS) for s in range(SHIFTS)) <= PART_TIME_SHIFTS_AT_MOST)
        model.Add(sum(x[e, d, s] for d in range(DAYS) for s in range(SHIFTS)) >= PART_TIME_SHIFTS_AT_LEAST)
    else:
        model.Add(sum(x[e, d, s] for d in range(DAYS) for s in range(SHIFTS)) <= FULL_TIME_SHIFTS_AT_MOST)
        model.Add(sum(x[e, d, s] for d in range(DAYS) for s in range(SHIFTS)) >= FULL_TIME_SHIFTS_AT_LEAST)

In [13]:
# Constraint 5: Each employee at least have one shift at friday or saturday evening unless she/he don't want to
for e in range(employees):
    if prefer_value[e][5][1] != -50 and prefer_value[e][6][1] != -50:
        model.Add((x[e, 5, 1] + x[e, 6, 1]) >= 1)

In [14]:
# Constraint 6: George only work at friday and saturday night.
for e in range(employees):
    if preferences["Employee Name"][e] == "George":
        model.Add(sum(x[e, d, s] for d in range(DAYS) for s in range(SHIFTS)) == 2)
        model.Add((x[e, 5, 1] + x[e, 6, 1]) == 2)

In [15]:
# Objective Function find total Maximum Happy points
model.Maximize(sum(prefer_value[e][d][s] * x[e,d,s] for e in range(employees) \
                   for d in range(DAYS) for s in range(SHIFTS)))

In [16]:
# Call Solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [18]:
# Output schedule to excel
schedule = {"%s %s" % (WEEKDAYS_DICTIONARY[d], SHIFTS_DICTIONARY[s]) : [] for d in range(DAYS) for s in range(SHIFTS)}
if status == cp_model.OPTIMAL:
    for e in range(employees):
        for d in range(DAYS):
            for s in range(SHIFTS):
                if solver.Value(x[e, d, s]) == 1 and prefer_value[e][d][s] == -50:
                    schedule[f"{WEEKDAYS_DICTIONARY[d]} {SHIFTS_DICTIONARY[s]}"].append(preferences["Employee Name"][e] + "(X)")
                elif solver.Value(x[e, d, s]) == 1:
                    schedule[f"{WEEKDAYS_DICTIONARY[d]} {SHIFTS_DICTIONARY[s]}"].append(preferences["Employee Name"][e])
                else:
                    schedule[f"{WEEKDAYS_DICTIONARY[d]} {SHIFTS_DICTIONARY[s]}"].append("-")
    df = pd.DataFrame(schedule, index=preferences["Employee Name"])
    df.to_excel("TapHouse_Optimal_schedule.xlsx", freeze_panes=(1, 0))
else:
    print("No optimal schedule found")